In [2]:
%pip install mediapipe opencv-python pandas scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [24]:
import mediapipe as mp
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [25]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# First step

In [4]:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    min_detection_confidence=0.5, min_tracking_confidence=0.5
) as pose:  # Setup mediapipe instance
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )
        
        cv2.imshow("Raw Webcam Feed", image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


# Save Video

In [14]:
cap = cv2.VideoCapture(0)

height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)
video_writer = cv2.VideoWriter('deadlift.avi', cv2.VideoWriter_fourcc('P', 'I', 'M', '1'), fps, (int(width), int(height)))

while cap.isOpened():
    ret, frame = cap.read()
    
    cv2.imshow('Press', frame)
    video_writer.write(frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    

cap.release()
# video_writer.release()
cv2.destroyAllWindows()

# Capture Landmarks & Export to CSV

In [24]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [59]:
landmarks = ['class']
for val in range(1, 33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [60]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [61]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten()
        keypoints = np.insert(keypoints.astype(object), 0, action)
        
        with open('coords.csv', mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        print(e)
        pass

In [62]:
cap = cv2.VideoCapture('deadlift.avi')
count = 0
with mp_pose.Pose(
    min_detection_confidence=0.5, min_tracking_confidence=0.5
) as pose:  # Setup mediapipe instance
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2),
        )
        
        k = cv2.waitKey(1)
        if k == 119:
            export_landmark(results, 'up')
            print("up")
        if k == 115:
            export_landmark(results, 'down')
            print("down")
        
        cv2.imshow("Raw Webcam Feed", image)
        count += 1

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

['up' 0.3780982196331024 0.5288584232330322 -0.24915459752082825
 0.9999650120735168 0.35949912667274475 0.49812284111976624
 -0.22950372099876404 0.9999523162841797 0.3608241379261017
 0.49519020318984985 -0.2297997772693634 0.9999578595161438
 0.3618974983692169 0.49261024594306946 -0.23008546233177185
 0.9999565482139587 0.34760698676109314 0.5134125351905823
 -0.2626400291919708 0.9999476075172424 0.34024375677108765
 0.5220858454704285 -0.2624231278896332 0.9999529719352722
 0.3323673903942108 0.5317336320877075 -0.2625271677970886
 0.9999412894248962 0.34543198347091675 0.5196495652198792
 -0.07971478998661041 0.9999660849571228 0.3122503161430359
 0.5753970742225647 -0.21831710636615753 0.9999435544013977
 0.38790881633758545 0.5668039321899414 -0.18331344425678253
 0.9999407529830933 0.37623313069343567 0.5868001580238342
 -0.22464296221733093 0.9999139308929443 0.3916926681995392
 0.6754156947135925 0.0740910992026329 0.9986153841018677
 0.37722694873809814 0.757298469543457 -

# Train Custom Model Using Scikit Learn

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('coords.csv')

In [72]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.378098,0.528858,-0.249155,0.999965,0.359499,0.498123,-0.229504,0.999952,0.360824,...,-0.020554,0.015343,0.798491,2.104872,0.115061,0.043989,0.721570,2.222583,-0.276448,0.030895
1,up,0.382003,0.524666,-0.241485,0.999962,0.362467,0.495825,-0.216278,0.999948,0.362984,...,0.033239,0.016480,0.833061,2.050314,0.219360,0.040606,0.766536,2.183540,-0.195772,0.030238
2,up,0.382638,0.520930,-0.246137,0.999958,0.363607,0.493407,-0.225828,0.999944,0.363880,...,0.053926,0.016454,0.862108,2.055442,0.233927,0.036631,0.788270,2.172866,-0.181902,0.028497
3,up,0.382474,0.514313,-0.257925,0.999948,0.363114,0.489805,-0.236498,0.999932,0.362991,...,0.052699,0.016450,0.877936,2.024809,0.244526,0.033619,0.805597,2.124985,-0.187544,0.026976
4,up,0.383339,0.513099,-0.245043,0.999918,0.363981,0.488349,-0.225018,0.999892,0.363739,...,0.049383,0.014714,0.913402,1.993801,0.232849,0.028808,0.857152,2.075278,-0.191954,0.023724


In [73]:
df.tail()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
34,up,0.372389,0.493314,-0.145452,0.996830,0.352778,0.470333,-0.125897,0.997341,0.352425,...,0.097900,0.254474,0.968203,0.859330,0.357575,0.107538,1.020118,1.032453,0.105959,0.201335
35,up,0.376033,0.490771,-0.081462,0.996620,0.356967,0.469096,-0.049099,0.996351,0.357086,...,0.080124,0.279861,1.003806,1.346459,0.444246,0.095589,1.032596,1.365378,0.062640,0.215214
36,up,0.376176,0.491889,-0.087435,0.996518,0.356727,0.469444,-0.057685,0.995611,0.356812,...,0.115017,0.297498,1.012085,1.370546,0.464388,0.081784,1.050682,1.427591,0.056963,0.219924
37,up,0.374172,0.455773,-0.132654,0.994177,0.351393,0.438748,-0.109988,0.992620,0.351363,...,0.111571,0.288494,1.051930,1.182170,0.420456,0.076447,1.085475,1.307531,0.088957,0.216648
38,up,0.408455,0.426737,-0.054868,0.994120,0.387075,0.409593,-0.026889,0.992716,0.385871,...,0.030192,0.282123,1.061215,1.181332,0.355025,0.074455,1.088153,1.305111,-0.016936,0.207436


In [75]:
df[df['class'] == 'up']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,up,0.378098,0.528858,-0.249155,0.999965,0.359499,0.498123,-0.229504,0.999952,0.360824,...,-0.020554,0.015343,0.798491,2.104872,0.115061,0.043989,0.721570,2.222583,-0.276448,0.030895
1,up,0.382003,0.524666,-0.241485,0.999962,0.362467,0.495825,-0.216278,0.999948,0.362984,...,0.033239,0.016480,0.833061,2.050314,0.219360,0.040606,0.766536,2.183540,-0.195772,0.030238
2,up,0.382638,0.520930,-0.246137,0.999958,0.363607,0.493407,-0.225828,0.999944,0.363880,...,0.053926,0.016454,0.862108,2.055442,0.233927,0.036631,0.788270,2.172866,-0.181902,0.028497
3,up,0.382474,0.514313,-0.257925,0.999948,0.363114,0.489805,-0.236498,0.999932,0.362991,...,0.052699,0.016450,0.877936,2.024809,0.244526,0.033619,0.805597,2.124985,-0.187544,0.026976
4,up,0.383339,0.513099,-0.245043,0.999918,0.363981,0.488349,-0.225018,0.999892,0.363739,...,0.049383,0.014714,0.913402,1.993801,0.232849,0.028808,0.857152,2.075278,-0.191954,0.023724
9,up,0.380362,0.489997,-0.114222,0.998523,0.360560,0.466749,-0.100598,0.999014,0.360678,...,0.060924,0.063248,0.911275,1.885740,0.265977,0.077134,0.896488,1.988878,-0.126015,0.057385
10,up,0.376395,0.491188,-0.155249,0.998676,0.356786,0.468019,-0.139213,0.999071,0.357025,...,0.109378,0.072315,0.946137,1.777773,0.366766,0.079783,0.930577,1.879747,-0.021140,0.067197
11,up,0.376450,0.500396,-0.202934,0.998936,0.357358,0.478432,-0.186286,0.999179,0.357291,...,0.024553,0.120882,0.932920,1.712007,0.346645,0.107177,0.922421,1.843300,-0.116000,0.116888
12,up,0.379059,0.503257,-0.156279,0.999008,0.359736,0.481285,-0.133428,0.999148,0.359223,...,0.088685,0.108234,0.947943,1.735454,0.342117,0.104203,0.927041,1.817746,-0.092284,0.106191
13,up,0.381487,0.503530,-0.118817,0.998740,0.362846,0.481150,-0.090599,0.998808,0.362462,...,0.132337,0.069858,1.004989,1.655370,0.352772,0.071832,0.982428,1.743934,-0.042896,0.072655


In [3]:
x = df.drop('class', axis=1)
y = df['class']

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1234)

In [5]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [6]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

In [7]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(x_train, y_train)
    fit_models[algo] = model

In [8]:
fit_models['rc'].predict(x_test)

array(['down', 'up', 'up', 'up', 'up', 'down', 'down', 'down', 'up',
       'down', 'up', 'down'], dtype='<U4')

# Evaluate and Serialize Model

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import pickle

In [10]:
for algo, model in fit_models.items():
    yhat = model.predict(x_test)
    print(algo, accuracy_score(y_test.values, yhat),
          precision_score(y_test.values, yhat, average="binary", pos_label="up"),
          recall_score(y_test.values, yhat, average="binary", pos_label="up"))

lr 0.75 0.6666666666666666 1.0
rc 1.0 1.0 1.0
rf 0.8333333333333334 0.75 1.0
gb 1.0 1.0 1.0


In [17]:
yhat = fit_models['rf'].predict(x_test)

In [20]:
yhat[:10]

array(['down', 'up', 'up', 'up', 'up', 'down', 'down', 'down', 'up',
       'down'], dtype=object)

In [19]:
y_test

8     down
38      up
14      up
1       up
4       up
31    down
20    down
17    down
13      up
21    down
0       up
7     down
Name: class, dtype: object

In [21]:
with open('deadlift.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Make Detections with Model

In [32]:
import warnings
warnings.filterwarnings('ignore')

In [22]:
with open('deadlift.pkl', 'rb') as f:
    model = pickle.load(f)

In [39]:
cap = cv2.VideoCapture(0)

counter = 0
current_stage = None

with mp_pose.Pose(
    min_detection_confidence=0.5, min_tracking_confidence=0.5
) as pose:  # Setup mediapipe instance
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(245, 117, 66),
                                   thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(245, 66, 230),
                                   thickness=2, circle_radius=2),
        )

        try:
            row = np.array(
                [
                    [res.x, res.y, res.z, res.visibility]
                    for res in results.pose_landmarks.landmark
                ]
            ).flatten()
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            
            if body_language_class == 'down' and body_language_prob[body_language_prob.argmax()] <= .65:
                current_stage = 'down'
            elif current_stage == 'down' and body_language_class == 'up' and body_language_prob[body_language_prob.argmax()] > .65:
                print("hola")
                current_stage = 'up'
                counter += 1

            cv2.rectangle(image, (0, 0), (250, 60), (245, 117, 16), -1)

            cv2.putText(
                image,
                "CLASS",
                (95, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                body_language_class.split(" ")[0],
                (90, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

            cv2.putText(
                image,
                "PROB",
                (15, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                str(round(
                    body_language_prob[np.argmax(body_language_prob)], 2)),
                (10, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )
            
            cv2.putText(
                image,
                "COUNT",
                (180, 12),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                1,
                cv2.LINE_AA,
            )
            cv2.putText(
                image,
                str(round(
                    counter)),
                (175, 40),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                (255, 255, 255),
                2,
                cv2.LINE_AA,
            )

        except Exception as e:
            pass

        cv2.imshow("Raw Webcam Feed", image)

        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

    cap.release()
    cv2.destroyAllWindows()

OpenCV: out device of bound (0-0): 1
OpenCV: camera failed to properly initialize!


: 